In [25]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import us
from us import states
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

# Census API Key
from config import api_key
c = Census(api_key, year=2016)

In [17]:
# Using census api to pull fields of interest.  Codes below are from Census website
census_data=c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B02001_002E", "B02001_003E", "B02001_004E","B02001_005E","B02001_006E","B02001_007E","B02001_008E", "B02001_009E", "B02001_010E", "B01002_001E",
                          "B19301_001E","B23025_001E","B23025_002E", "B23025_005E",
                          "B17001_002E", "B01001_001E", "B01001_002E","B01001_026E", "B01001_007E", "B01001_008E", "B01001_009E", "B01001_010E", "B01001_011E","B01001_012E", "B01001_013E", "B01001_014E", "B01001_015E",
                       "B01001_016E", "B01001_017E", "B01001_018E", "B01001_019E", "B01001_020E", "B01001_021E", "B01001_022E","B01001_023E", "B01001_024E","B01001_025E",
                       "B01001_031E", "B01001_032E", "B01001_033E", "B01001_034E", "B01001_035E","B01001_036E", "B01001_037E", "B01001_038E", "B01001_039E",
                       "B01001_040E", "B01001_041E", "B01001_042E", "B01001_043E", "B01001_044E", "B01001_045E", "B01001_046E","B01001_047E", "B01001_048E","B01001_049E",
                       "B05001_001E", "B05001_006E"), {'for': 'state:*'})

In [18]:
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Renaming
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B02001_002E": "White alone", 
                                      "B02001_003E": "Black or African American alone", 
                                      "B02001_004E": "American Indian and Alaska Native alone",
                                      "B02001_005E": "Asian alone",
                                      "B02001_006E": "Native Hawaiian and Other Pacific Islander alone",
                                      "B02001_007E": "Some other race alone",
                                      "B02001_008E": "Two or more races", 
                                      "B02001_009E": "Two races including Some other race", 
                                      "B02001_010E": "Two races excluding Some other race and three or more races",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_001E": "Total employable population, over 16",
                                      "B23025_002E": "Total in labor force",
                                      "B23025_005E": "Civilian labor force Unemployed",
                                      "B01001_001E": "Population Total count by sex",
                                      "B01001_002E": "Total Male",
                                      "B01001_026E": "Total Female",
                                      "B01001_007E": "Male 18-19",
                                      "B01001_008E": "Male 20", 
                                      "B01001_009E": "Male 21", 
                                      "B01001_010E": "Male 22-24", 
                                      "B01001_011E": "Male 25-29",
                                      "B01001_012E": "Male 30-34", 
                                      "B01001_013E": "Male 35-39", 
                                      "B01001_014E": "Male 40-44", 
                                      "B01001_015E": "Male 45-49",
                                      "B01001_016E": "Male 50-54",
                                      "B01001_017E": "Male 55-59", 
                                      "B01001_018E": "Male 60-61", 
                                      "B01001_019E": "Male 62-64", 
                                      "B01001_020E": "Male 65-66", 
                                      "B01001_021E": "Male 67-69", 
                                      "B01001_022E": "Male 70-74",
                                      "B01001_023E": "Male 75-79", 
                                      "B01001_024E": "Male 80-84",
                                      "B01001_025E": "Male 85 and up",
                                      "B01001_031E": "Male 18-19",
                                      "B01001_032E": "Female 20", 
                                      "B01001_033E": "Female 21", 
                                      "B01001_034E": "Female 22-24", 
                                      "B01001_035E": "Female 25-29",
                                      "B01001_036E": "Female 30-34", 
                                      "B01001_037E": "Female 35-39", 
                                      "B01001_038E": "Female 40-44", 
                                      "B01001_039E": "Female 45-49",
                                      "B01001_040E": "Female 50-54",
                                      "B01001_041E": "Female 55-59", 
                                      "B01001_042E": "Female 60-61", 
                                      "B01001_043E": "Female 62-64", 
                                      "B01001_044E": "Female 65-66", 
                                      "B01001_045E": "Female 67-69", 
                                      "B01001_046E": "Female 70-74",
                                      "B01001_047E": "Female 75-79", 
                                      "B01001_048E": "Female 80-84",
                                      "B01001_049E": "Female 85 and up",
                                      "B05001_001E": "Citizenship population count citizen and non-citizen",
                                      "B05001_006E": "Non Citizen",
                                      "NAME": "Name"})

# Add in Poverty Rate, Unemployement Rate, and citzenship rates.
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

census_pd["Unemployement Rate"] = 100 * census_pd["Civilian labor force Unemployed"].astype(int) / census_pd["Total employable population, over 16"].astype(int)
census_pd["Percent Non-Citizen"] = 100 * census_pd["Non Citizen"] / census_pd["Citizenship population count citizen and non-citizen"]
census_pd["% Citizenship"] = 100 - census_pd["Percent Non-Citizen"]

In [19]:
# View results for each state
census_pd.head(50)

,Name,Household Income,Population,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,...,Female 70-74,Female 75-79,Female 80-84,Female 85 and up,Citizenship population count citizen and non-citizen,Non Citizen,Poverty Rate,Unemployement Rate,Percent Non-Citizen,% Citizenship
0,Alabama,44758.0,4841164.0,3325037.0,1282053.0,23919.0,60744.0,2008.0,61991.0,85412.0,...,100123.0,76173.0,55764.0,55840.0,4841164.0,106663.0,17.943329,4.773928,2.203251,97.796749
1,Alaska,74444.0,736855.0,483518.0,24443.0,103574.0,44218.0,8862.0,9900.0,62340.0,...,9041.0,4957.0,3701.0,3872.0,736855.0,24109.0,9.883356,5.299333,3.271878,96.728122
2,Arizona,51340.0,6728577.0,5235158.0,287110.0,296732.0,205229.0,12863.0,468906.0,222579.0,...,143438.0,102397.0,71555.0,75848.0,6728577.0,536203.0,17.323663,4.724619,7.969040,92.030960
3,Arkansas,42336.0,2968472.0,2307136.0,460638.0,18697.0,41539.0,6738.0,67645.0,66079.0,...,62998.0,44963.0,34675.0,36391.0,2968472.0,94459.0,18.273071,3.983658,3.182075,96.817925
4,California,63783.0,38654206.0,23680584.0,2261835.0,285512.0,5354608.0,150908.0,5133600.0,1787159.0,...,630402.0,468253.0,364898.0,437219.0,38654206.0,5298355.0,15.533257,5.508539,13.707059,86.292941
5,Colorado,62520.0,5359295.0,4517058.0,219349.0,50160.0,157523.0,7441.0,224029.0,183735.0,...,85262.0,61476.0,44794.0,51038.0,5359295.0,317762.0,11.903394,4.044108,5.929175,94.070825
6,Connecticut,71755.0,3588570.0,2768080.0,372696.0,9399.0,152782.0,1031.0,177594.0,106988.0,...,68489.0,51857.0,42732.0,59327.0,3588570.0,254629.0,10.044781,5.323111,7.095556,92.904444
7,Delaware,61017.0,934695.0,647002.0,203027.0,3494.0,34428.0,461.0,20452.0,25831.0,...,20363.0,14926.0,10278.0,11622.0,934695.0,43979.0,11.709488,4.471083,4.705171,95.294829
8,District of Columbia,72935.0,659009.0,266035.0,318598.0,2174.0,24036.0,271.0,29650.0,18245.0,...,9981.0,7838.0,5507.0,7275.0,659009.0,53982.0,17.004320,6.007066,8.191390,91.808610
9,Florida,48900.0,19934451.0,15130748.0,3216994.0,52904.0,521272.0,11288.0,505877.0,495368.0,...,493807.0,373731.0,283192.0,322646.0,19934451.0,1832945.0,15.747903,4.908509,9.194861,90.805139


In [21]:
#  Export dataframe to sql database.  Also print out to view all data and db.

conn = 'sqlite:///voting.db'
engine = create_engine(conn, if_exists='replace', echo=False)

sample_sql_database = census_pd.to_sql('sample_database', con=engine)

sample_sql_database = engine.execute("SELECT * FROM sample_database").fetchall()

print(sample_sql_database)

[(0, 'Alabama', 44758.0, 4841164.0, 3325037.0, 1282053.0, 23919.0, 60744.0, 2008.0, 61991.0, 85412.0, 6877.0, 78535.0, 38.6, 24736.0, 3864302.0, 2238654.0, 184479.0, 868666.0, '4841164', 2346193.0, 2494971.0, 64288.0, 37110.0, 36861.0, 97495.0, 157445.0, 150718.0, 142795.0, 153127.0, 153538.0, 165372.0, 159393.0, 57496.0, 83307.0, 51203.0, 65750.0, 83372.0, 56930.0, 36796.0, 26381.0, 35875.0, 36158.0, 98990.0, 161732.0, 156669.0, 152831.0, 157463.0, 161471.0, 176258.0, '01', 172841.0, 65434.0, 91124.0, 58864.0, 75198.0, 100123.0, 76173.0, 55764.0, 55840.0, 4841164.0, 106663.0, 17.943329331540927, 4.773928124665205, 2.2032511189457744, 97.79674888105423), (1, 'Alaska', 74444.0, 736855.0, 483518.0, 24443.0, 103574.0, 44218.0, 8862.0, 9900.0, 62340.0, 3056.0, 59284.0, 33.6, 34191.0, 568732.0, 400475.0, 30139.0, 72826.0, '736855', 385296.0, 351559.0, 8210.0, 6454.0, 6537.0, 20685.0, 32384.0, 28830.0, 25203.0, 22825.0, 24496.0, 27423.0, 26562.0, 9739.0, 12593.0, 7308.0, 7532.0, 9156.0, 5268